In [1]:
import pandas as pd
import os
import sys
import zipfile
import json
from tqdm.auto import tqdm
tqdm.pandas()
import re
import pickle
util_dir = os.path.abspath('../utils')
sys.path.append(util_dir)
from utils import *

C:\Users\Niek\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
file = 'csv/treasury_shoes2.txt'
tr_df = pd.read_csv(file, encoding='utf8')

In [3]:
ids = list(tr_df['id_text'])
ids.sort()

In [4]:
project = 'epsd2/admin/ur3'
#oracc_download([project])

In [5]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject)
        if "label" in JSONobject:
            meta_d["label"] = JSONobject['label']
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            if "ftype" in JSONobject:
                lemma['ftype'] = JSONobject['ftype'] # this picks up YN for year name
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in dollar_keys}
            lemma["id_word"] = JSONobject["ref"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
    return

In [6]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
dollar_keys = ["extent", "scope", "state"]
file = f"jsonzip/{project.replace('/', '-')}.zip"
try:
    z = zipfile.ZipFile(file) 
except:
    print(f"{file} does not exist or is not a proper ZIP file")
files = z.namelist()
files = [name for name in files if name[-12:-5] in ids]
for filename in tqdm(files, desc = project):
    id_text = project + filename[-13:-5] 
    meta_d["id_text"] = id_text
    #try:
    st = z.read(filename).decode('utf-8')
    data_json = json.loads(st)           
    parsejson(data_json)
    #except:
    #print(f'{id_text} is not available or not complete')
z.close()

In [7]:
words = pd.DataFrame(lemm_l).fillna('')
keep = ['extent', 'scope', 'state', 'id_word', 'id_text', 'form', 'cf', 'gw', 'pos']
words = words[keep]

In [8]:
findreplace = {' ' : '-', ',' : ''}
words = words.replace({'gw' : findreplace, 'sense' : findreplace}, regex=True)

In [9]:
words['id_text'] = [i[-7:] for i in words['id_text']]
words['id_line'] = [int(i.split('.')[1]) for i in words['id_word']]

In [10]:
proper_nouns = ['FN', 'PN', 'DN', 'AN', 'WN', 'ON', 'TN', 'MN', 'CN', 'GN']
physical_break = ['illegible', 'traces', 'missing', 'effaced']
logical_break = ['other', 'blank', 'ruling']
words['lemma'] = words["cf"] + '[' + words["gw"] + ']' + words["pos"]
words.loc[words["cf"] == "" , 'lemma'] = words['form'] + '[NA]NA'
words.loc[words["pos"] == "n" , 'lemma'] = words['form'] + '[]NU'
words["lemma"] = words.progress_apply(lambda r: f"{r['lemma'][:-5]}1]{r['pos']}" 
                            if r["pos"] in proper_nouns else r['lemma'], axis=1)
words.loc[words["state"].isin(logical_break), 'lemma'] = "break_logical"
words.loc[words["state"].isin(physical_break), 'lemma'] = "break_physical"
words.head(10)

,extent,scope,state,id_word,id_text,form,cf,gw,pos,id_line,lemma
0,n,line,missing,P104232.3,P104232,,,,,3,break_physical
1,,,,P104232.5.1,P104232,šu,šu,hand,N,5,šu[hand]N
2,,,,P104232.5.2,P104232,ba-ti,teŋ,approach,V/i,5,teŋ[approach]V/i
3,,,,P104232.6.1,P104232,ša₃,šag,heart,N,6,šag[heart]N
4,,,,P104232.6.2,P104232,puzur₄-iš-{d}da-gan,,,GN,6,puzur₄-iš-{d}da-gan[1]GN
5,,,,P104232.7.1,P104232,iti,itud,moon,N,7,itud[moon]N
6,,,,P104232.7.2,P104232,ki-siki-{d}nin-a-zu,Kisikininazuk,1,MN,7,Kisikininazuk1]MN
7,,,,P104232.7.3,P104232,min-kam,min,two,NU,7,min[two]NU
8,,,,P104232.8.1,P104232,mu,mu,year,N,8,mu[year]N
9,,,,P104232.8.2,P104232,si-mu-ru-um{ki},,,GN,8,si-mu-ru-um{ki}[1]GN


In [11]:
normdf = pd.read_csv('Normalized/drehem_norm_names.csv', encoding='utf8')
normdf

,transliteration,normalization,remarks,atftr,sign_names
0,A-AN-ba-az,A.AN.ba.az[]PN,MVN 13 464 r 10 (copy/photo),A-AN-ba-az,A AN BA |PIRIG×ZA|
1,A-KU-um,A.KU.um[]PN,"Aegyptus 10, 270 27 o 7 (copy)",A-KU-um,A KU UM
2,A-KU.KU-ta,A.KU.KU[]PN,AnOr 12 277 o i 17' (copy),A-KU.KU-ta,A KU KU TA
3,A-NI-ta,A.NI[]PN,Babyl. 7 pl. 22 18 o 3 (copy),A-NI-ta,A NI TA
4,A-U.E₂-nu-tuku,A.U.E₂.nu.tuku[]PN,AnOr 07 150 o 2: A-U.KID-nu-tuku IŠ (copy/photo),A-U.E₂-nu-tuku,A U E₂ NU TUK
...,...,...,...,...,...
5785,{d}nin-sun₂,Ninsumunak[]DN,NaN,{d}nin-sun₂,AN SAL TUG₂ GUL
5786,{d}nin-tin-ug₅-ga,Nintiluga[]DN,NaN,{d}nin-tin-ug₅-ga,AN SAL TUG₂ DIN |EZEN×BAD| GA
5787,{d}nin-urta,Ninurta[]DN,NaN,{d}nin-urta,AN SAL TUG₂ IB
5788,{d}utu,Utu[]DN,NaN,{d}utu,AN UD


In [12]:
letters = r'a-zḫĝŋṣšṭA-ZḪĜŊṢŠṬ'
lettersX = re.compile(fr'(?<=[{letters}])x') # X preceded by a letter
lettersNo = re.compile(fr'[{letters}](\d+|x)') # any sequence of digits, or X, preceded by a letter
xv = re.compile(fr'[\w]+x') #this matches a sequence of word signs (letters) and/or flags, followed by X
xvalues = {'nagx' : '₃', 'nigarx' : '', 'nemurx' : '₂', 'pešx' : '₁₄', 'urubx' : '', 
        'tubax' : '₄', 'niginx' : '₈', 'šux' : '₁₄', 
        'alx' : 'ₓ(|NUN.LAGAR|)' , 'bulugx' : 'ₓ(|ŠIM×KUŠU₂|)', 'dagx' : 'ₓ(KWU844)', 
        'durux' : 'ₓ(|IGI.DIB|)', 'durunx' : 'ₓ(|KU.KU)', 
        'gigirx' : 'ₓ(|LAGAB×MU|)', 'giparx' : 'ₓ(KISAL)', 'girx' : 'ₓ(GI)', 
        'gišbunx' : 'ₓ(|KI.BI|)', 'gurx' : 'ₓ(|ŠE.KIN|)', 
        'hirinx' : 'ₓ(KWU318)', 'kurunx' : 'ₓ(|DIN.BI|)',
        'mu-kux' : 'ₓ(DU)', 'munsubx' : 'ₓ(|PA.GU₂×NUN|)',  
        'sagx' : 'ₓ(|ŠE.KIN|)', 'subx' : 'ₓ(|DU.DU|)', 
        'sullimx' : 'ₓ(EN)', 'šaganx' : 'ₓ(|GA×AN.GAN|)', 
        'ulušinx' : 'ₓ(|BI.ZIZ₂|)', 'zabalamx' : 'ₓ(|MUŠ₃.TE.AB@g|)', 
        'zahx' : 'ₓ(ŠEŠ)', 'zahdax' : 'ₓ(|DUN.NE.TUR|)',  
        'zix' : 'ₓ(IGI@g)'}

In [13]:
def ogsl_v(text):
    # 1. deal with unambiguous x-values, listed in the dictionary xvalues.
    ogsl_valid = re.sub(xv, lambda m: m.group()[:-1] + xvalues.get(m.group().lower(), 'x'), text)
    return ogsl_valid

In [14]:
normdf["atftr"] = normdf['transliteration'].progress_map(ogsl_v) 

In [15]:
oracc_download(['ogsl'])

Saving http://build-oracc.museum.upenn.edu/json/ogsl.zip as jsonzip/ogsl.zip.


['ogsl']

In [16]:
equiv = {'ANŠE' : 'GIR₃', 
        'DUR₂' : 'KU', 
        'NAM₂' : 'TUG₂', 
        'TIL' : 'BAD', 
        'NI₂' : 'IM',
        'ŠAR₂' : 'HI', 
        }
w = re.compile(r'\w+') # replace whole words only - do not replace TILLA with BADLA.
           # but do replace |SAL.ANŠE| with |SAL.GIR₃|

In [17]:
def parsejson(data_json):
    for key, value in data_json["signs"].items():
        key = re.sub(w, lambda m: equiv.get(m.group(), m.group()), key)
        if "values" in value:
            for n in value["values"]:
                d2[n] = key
    return

In [18]:
d2 = {}  # this empty dictionary is filled by the parsejson() function, called in this cell.
file = "jsonzip/ogsl.zip"
z = zipfile.ZipFile(file) 
filename = "ogsl/ogsl-sl.json"
signlist = z.read(filename).decode('utf-8')
data_json = json.loads(signlist)                # make it into a json object (essentially a dictionary)
parsejson(data_json)  
with open('output/ogsl_dict.p', 'wb') as p:
    pickle.dump(d2, p)  

In [19]:
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']  # used in compound signs
#operators = ['&', '%', '@', '×']
flags = "][?<>⸢⸣⌈⌉*/" # note that ! is omitted from flags, because it is dealt with separately
table = str.maketrans(dict.fromkeys(flags))

In [20]:
def signnames(translit):  
    """This function takes a string of transliterated cuneiform text and translates that string into a string of
    sign names, separated by spaces. In order to work it needs the variables separators, separators2, and table defined above. The variable table
    is used by the translate() method to translate all flags (except for !) to None. The function also needs a dictionary, called d2, that has as
    keys sign readings and sign names as corresponding values. In case a key is not found, the sign reading is replaced by itself."""
    signnames_l = []
    translit = translit.translate(table).lower()  # remove flags, half brackets, square brackets.
    translit = translit.replace('...', 'x')
    for s in separators: # split transliteration line into signs   
        translit = translit.replace(s, ' ').strip()
    s_l = translit.split() # s_l is a list that contains the sequence of transliterated signs without separators or flags
    s_l = [d2.get(sign, sign) for sign in s_l] # replace each transliterated sign with its sign name.
    # Now take care of some special situations: signs with qualifiers, compound signs.
    for sign in s_l:
        if '!' in sign: # corrected sign, as in ka!(SAG), get only the corrected reading.
            sign = sign.split('!(')[0]
            sign = sign.replace('!', '') # remove remaining exclamtion marks
        elif sign[-1] == ')' and '(' in sign: # qualified sign, as in ziₓ(SIG₇) - get only the qualifier
            sign = sign.split('(')[1][:-1]
        if '×' in sign: #compound. Compound like |KA×NINDA| to be replaced by |KA×GAR|
            sign_l = sign.replace('|', '').split('×')
            #replace individual signs of the compound by OGSL names
            sign_l = [d2.get(sign, sign) for sign in sign_l] 
            # if user enters |KA*EŠ| this is transformed to ['KA', '|U.U.U|']. The pipes around U.U.U must be replaced by brackets
            sign_l = [f'({sign[1:-1]})' if len(sign) > 1 and sign[0] == '|' else sign for sign in sign_l]
            sign = f"|{'×'.join(sign_l)}|"  #put the sign together again with enclosing pipes.
        elif '.' in sign or '+' in sign: # using elif, so that compounds like |UD×(U.U.U)| are not further analyzed.
            for s in separators2:
                sign = sign.replace(s, ' ').strip() 
            sign_l = sign.split()  # compound sign split into multiple signs
            sign_l = [d2.get(sign, sign) for sign in sign_l]
            signnames_l.extend(sign_l)
            continue
        sign = d2.get(sign, sign)
        signnames_l.append(sign)
    # add space before and after each line so that each sign representation is enclosed in spaces
    signnames = f" {' '.join(signnames_l).upper()} " 
    return signnames

In [21]:
normdf["sign_names"] = normdf["atftr"].progress_map(signnames)

In [22]:
normd2 = dict(zip(normdf['sign_names'], normdf['normalization']))

In [23]:
words['sign_names'] = words['form'].progress_map(signnames)

In [24]:
words['name_norm'] = words.sign_names.progress_apply(lambda x: normd2.get(x, ''))

In [25]:
words.loc[(words.pos == 'PN') & (words.name_norm == '')]

,extent,scope,state,id_word,id_text,form,cf,gw,pos,id_line,lemma,sign_names,name_norm
24,,,,P103277.7.2,P103277,sipa-si-ni,,,PN,7,sipa-si-ni[1]PN,PA LU SI NI,
152,,,,P124610.5.3,P124610,lugal-ka,,,PN,5,lugal-ka[1]PN,LUGAL KA,
239,,,,P143238.5.4,P143238,dingir-su₂-lagab-me-tur,,,PN,5,dingir-su₂-lagab-me-tur[1]PN,AN ZU LAGAB ME TUR,
382,,,,P113021.14.2,P113021,ma-lu-um,,,PN,14,ma-lu-um[1]PN,MA LU UM,
422,,,,P103308.4.2,P103308,a₂-an-za,,,PN,4,a₂-an-za[1]PN,A₂ AN ZA,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10828,,,,P291956.4.4,P291956,dingir-ka,,,PN,4,dingir-ka[1]PN,AN KA,
10950,,,,P144356.12.2,P144356,en-{d}nanna-{d}amar-{d}suen-ra-ki-ag₂-an-na,,,PN,12,en-{d}nanna-{d}amar-{d}suen-ra-ki-ag₂-an-na[1]PN,EN AN ŠEŠ KI AN AMAR AN EN ZU RA KI |NINDA₂×N...,
11194,,,,P200026.5.2,P200026,ṣe-eh-ru-um,,,PN,5,ṣe-eh-ru-um[1]PN,ZE₂ |HI×NUN| RU UM,
11238,,,,P125628.5.2,P125628,a-hu-nu-ta,,,PN,5,a-hu-nu-ta[1]PN,A HU NU TA,


In [29]:
failed = words.loc[(words.pos == 'PN') & (words.name_norm == ''), ['id_word', 'lemma', 'form']]

In [30]:
failed

,id_word,lemma,form
24,P103277.7.2,sipa-si-ni[1]PN,sipa-si-ni
152,P124610.5.3,lugal-ka[1]PN,lugal-ka
239,P143238.5.4,dingir-su₂-lagab-me-tur[1]PN,dingir-su₂-lagab-me-tur
382,P113021.14.2,ma-lu-um[1]PN,ma-lu-um
422,P103308.4.2,a₂-an-za[1]PN,a₂-an-za
...,...,...,...
10828,P291956.4.4,dingir-ka[1]PN,dingir-ka
10950,P144356.12.2,en-{d}nanna-{d}amar-{d}suen-ra-ki-ag₂-an-na[1]PN,en-{d}nanna-{d}amar-{d}suen-ra-ki-ag₂-an-na
11194,P200026.5.2,ṣe-eh-ru-um[1]PN,ṣe-eh-ru-um
11238,P125628.5.2,a-hu-nu-ta[1]PN,a-hu-nu-ta


In [32]:
failed.to_csv('Normalized/corrections.csv', encoding='utf8', index=False)

In [35]:
normdf = normdf[['transliteration', 'normalization', 'remarks']]
normdf = normdf.fillna('')

In [36]:
normdf.to_csv('Normalized/drehem_norm_names.csv', encoding='utf8', index=False)